In [20]:
from audioop import reverse
import time
from functools import reduce
from multiprocessing import Pool, cpu_count, Manager


n_cpus = cpu_count()
# word_dict = dict()


def clean_word(word):
    word = word.lower()
    word = word.replace(',', '')
    word = word.replace('.', '')
    word = word.replace('\\n', '')
    word = word.replace('\\t', '')
    word = word.replace('!', '')
    word = word.replace('?', '')
    if word.startswith('\''):
        word = word[1:]
    if word.endswith('\''):
        word = word[:-1]
    return word


def mapper(word):
    global word_dict

    if word not in word_dict:
        word_dict[word] = 1
    
    return word, 1


def reducer(a, b):
    global word_dict

    if a[0] == b[0]:
        count = a[1] + b[1]
        word_dict[a[0]] = word_dict[a[0]] + count
        return a
    return b


def chunkify(data, n_chunks):
    data = data.split()
    data = sorted(data)
    return (data[i:len(data):n_chunks] for i in range(n_chunks))


def chunks_mapper(chunk):
    global word_dict

    chunk = map(clean_word, chunk)
    mapped_chunk = map(mapper, chunk)
    return reduce(reducer, mapped_chunk)
    

with open('./opowiesc.txt', 'r') as f:
    text = f.read()
    text = text.split()
    text = [word.replace('–', ' ') if word.count('–') > 0 else word for word in text]
    text = " ".join(text)

start = time.process_time()
data_chunks = chunkify(text, n_cpus)


manager = Manager()
word_dict = manager.dict()

with Pool(processes=n_cpus) as pool:
    mapped = pool.map(chunks_mapper, data_chunks)
    reduced = reduce(reducer, mapped)

print(dict(sorted(word_dict.items(), key=lambda d:d[1], reverse=True)))
print(time.process_time() - start)

{'the': 73, 'of': 37, 'to': 37, 'you': 31, 'a': 31, 'and': 27, 'he': 27, 'in': 23, 'his': 21, 'was': 19, 'frank': 15, 'car': 15, 'it': 13, 'for': 13, 'diamond': 9, 'had': 9, 'said': 9, 'spencer': 7, 'on': 7, 'i': 5, 'young': 5, 'younger': 5, 'as': 3, 'with': 3, 'her': 3, 'police': 3, 'did': 1, "i'll": 1, 'hey': 1, "i've": 1, 'but': 1, 'have': 1, 'me': 1, 'are': 1, "i'm": 1, 'if': 1, 'relax': 1, 'hmmmm': 1, 'my': 1, 'is': 1, 'ok': 1, 'listen': 1, 'so': 1, 'imagine': 1, 'why': 1, 'just': 1, "that's": 1, "you're": 1, 'peggy': 1, 'oh': 1, 'they': 1, 'what': 1, 'nose': 1, 'would': 1, 'yes': 1, '0013': 1, '*****': 1, '0020': 1, "we'd": 1, '2330': 1, 'at': 1, 'after': 1, 'babbington': 1, "we've": 1, 'bahamas': 1, '0023': 1, 'babbingtons': 1, 'clive': 1, 'dog': 1, "don't": 1, 'duck': 1, '2337': 1, 'england': 1, 'five': 1, 'can': 1, "frank's": 1, 'half': 1, 'everyone': 1, "i'd": 1, 'inspector': 1, 'inside': 1, 'lane': 1, 'most': 1, 'hello': 1, 'outside': 1, 'ronald': 1, 'now': 1, 'sergeant': 1,